# 🚀 State-of-the-art Deep Learning: Transformers
**Source: TensorFlow in Action – Chapter 5**

Chapter 5 mengenalkan arsitektur **Transformer** untuk NLP, termasuk representasi teks sebagai angka, encoder–decoder, self-attention (termasuk masked dan multi-head), serta contoh implementasi blok Transformer dengan Keras sub-classing.


## 🔡 Representasi Teks sebagai Angka

**Theory**: Model deep learning membutuhkan input numerik, sehingga teks dikonversi menjadi urutan ID dan kemudian vektor (one-hot atau embedding).

- Tokenisasi → setiap kata diberi ID, misalnya `{"I":1, "went":2, ...}` sehingga kalimat jadi deret \([1,2,3,...]\).
- Untuk batch, panjang urutan diseragamkan dengan **padding** token khusus (misalnya ID 0) dan/atau **truncation**.
- Vectorisasi: tiap ID dipetakan ke vektor \(\mathbb{R}^d\), sehingga batch berbentuk tensor 3D `[batch, time, d]`.


In [1]:
import numpy as np

# kamus mini
vocab = {
    "<PAD>": 0,
    "I": 1,
    "went": 2,
    "to": 3,
    "the": 4,
    "beach": 5,
    "It": 6,
    "was": 7,
    "cold": 8,
}

sentences = [
    ["I", "went", "to", "the", "beach"],
    ["It", "was", "cold"],
]

max_len = 5

def encode_and_pad(sent, vocab, max_len):
    ids = [vocab[w] for w in sent]
    ids = ids[:max_len]
    ids += [0] * (max_len - len(ids))
    return ids

encoded = np.array([encode_and_pad(s, vocab, max_len) for s in sentences])
encoded


array([[1, 2, 3, 4, 5],
       [6, 7, 8, 0, 0]])

## 🔁 Arsitektur Encoder–Decoder Transformer

**Theory**: Transformer untuk machine translation memetakan kalimat sumber ke representasi laten dengan **encoder**, lalu menghasilkan kalimat target dengan **decoder**.

- Encoder: tumpukan layer yang masing-masing berisi *self-attention* + *feed-forward*.
- Decoder: tumpukan layer dengan *masked self-attention*, *encoder–decoder attention*, dan *feed-forward* untuk menghasilkan token demi token.


## 🎯 Self-Attention (Single Head)

**Theory**: Self-attention menghasilkan output pada setiap posisi sebagai kombinasi berbobot dari seluruh posisi lain dalam suatu urutan dengan menggunakan tiga proyeksi, yaitu Query \(Q\), Key \(K\), dan Value \(V\).

Diberikan input $$
X \in \mathbb{R}^{n \times d_{\text{model}}}
$$


$$
Q = X W_Q,\quad K = X W_K,\quad V = X W_V
$$

den,
$$
W_Q, W_K, W_V \in \mathbb{R}^{d_{\text{model}} \times d_k}
$$
\).

Skor perhatian:

$$
\text{scores} = \frac{Q K^\top}{\sqrt{d_k}}
$$

Probabilitas perhatian (per baris):

$$
P = \text{softmax}(\text{scores})
$$

Output self-attention:

$$
H = P V
$$


In [2]:
import tensorflow as tf
from tensorflow.keras import layers

class SelfAttention(layers.Layer):
    def __init__(self, d_model, **kwargs):
        super().__init__(**kwargs)
        self.d_model = d_model
    
    def build(self, input_shape):
        d_in = input_shape[-1]
        self.Wq = self.add_weight(
            shape=(d_in, self.d_model),
            initializer="glorot_uniform",
            trainable=True,
            name="Wq"
        )
        self.Wk = self.add_weight(
            shape=(d_in, self.d_model),
            initializer="glorot_uniform",
            trainable=True,
            name="Wk"
        )
        self.Wv = self.add_weight(
            shape=(d_in, self.d_model),
            initializer="glorot_uniform",
            trainable=True,
            name="Wv"
        )

    def call(self, x, mask=None):
        # x: [batch, time, d_in]
        Q = tf.matmul(x, self.Wq)  # [B, T, d_model]
        K = tf.matmul(x, self.Wk)
        V = tf.matmul(x, self.Wv)

        scores = tf.matmul(Q, K, transpose_b=True)  # [B, T, T]
        scores = scores / tf.math.sqrt(tf.cast(self.d_model, tf.float32))

        if mask is not None:
            # mask shape: [1, T, T] atau [B, T, T] (1 = keep, 0 = mask)
            scores = scores + (mask * -1e9)

        attn = tf.nn.softmax(scores, axis=-1)       # [B, T, T]
        out = tf.matmul(attn, V)                    # [B, T, d_model]
        return out, attn


## 🕶️ Masked Self-Attention di Decoder

**Theory**: Decoder menggunakan *masked self-attention* agar pada saat memprediksi token ke-\(t\), model tidak dapat melihat token setelah posisi \(t\) (*future tokens*).

Mask segitiga bawah didefinisikan sebagai:
$$
\text{mask}[i,j] =
\begin{cases}
1, & \text{jika } j > i \\
0, & \text{jika } j \le i
\end{cases}
$$

Mask ini ditambahkan ke skor perhatian sehingga posisi yang ter-*mask* memperoleh nilai yang sangat negatif dan menghasilkan probabilitas mendekati nol setelah fungsi *softmax*.


In [3]:
def causal_mask(seq_len):
    # 1 untuk posisi yang HARUS di-mask (future), 0 untuk yang boleh dilihat
    mask = tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    mask = 1.0 - mask  # upper triangle 1, lower triangle 0
    # ubah ke bentuk [1, T, T] agar bisa broadcast ke batch
    return tf.reshape(mask, (1, seq_len, seq_len))

T = 5
mask = causal_mask(T)
mask[0].numpy()


array([[0., 1., 1., 1., 1.],
       [0., 0., 1., 1., 1.],
       [0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0.]], dtype=float32)

## 🧠 Multi-Head Attention

**Theory**: Multi-head attention menjalankan beberapa self-attention secara paralel dengan dimensi yang lebih kecil, kemudian menggabungkan hasilnya.

Jika terdapat (h) head dan setiap head memiliki dimensi (d_head), maka:

$$
d_{\text{model}} = h \cdot d_{\text{head}}
$$

Output akhir multi-head attention dirumuskan sebagai:
$$
H = \text{Concat}(H^{(1)}, \dots, H^{(h)}) W_O
$$

dengan:
$$
W_O \in \mathbb{R}^{d_{\text{model}} \times d_{\text{model}}}
$$


In [5]:
class MultiHeadAttention(layers.Layer):
    def __init__(self, d_model, num_heads, **kwargs):
        super().__init__(**kwargs)
        assert d_model % num_heads == 0
        self.d_model = d_model
        self.num_heads = num_heads
        self.depth = d_model // num_heads
    
    def build(self, input_shape):
        d_in = input_shape[-1]
        self.Wq = self.add_weight(
            shape=(d_in, self.d_model),
            initializer="glorot_uniform",
            trainable=True,
            name="Wq"
        )
        self.Wk = self.add_weight(
            shape=(d_in, self.d_model),
            initializer="glorot_uniform",
            trainable=True,
            name="Wk"
        )
        self.Wv = self.add_weight(
            shape=(d_in, self.d_model),
            initializer="glorot_uniform",
            trainable=True,
            name="Wv"
        )
        self.Wo = self.add_weight(
            shape=(self.d_model, self.d_model),
            initializer="glorot_uniform",
            trainable=True,
            name="Wo"
        )

    def split_heads(self, x):
        # x: [B, T, d_model] → [B, num_heads, T, depth]
        B, T, _ = tf.unstack(tf.shape(x))
        x = tf.reshape(x, (B, T, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, x, mask=None):
        Q = tf.matmul(x, self.Wq)
        K = tf.matmul(x, self.Wk)
        V = tf.matmul(x, self.Wv)

        Q = self.split_heads(Q)
        K = self.split_heads(K)
        V = self.split_heads(V)

        # skor: [B, H, T, T]
        scores = tf.matmul(Q, K, transpose_b=True)
        scores = scores / tf.math.sqrt(tf.cast(self.depth, tf.float32))

        if mask is not None:
            # mask: [1, T, T] → broadcast ke [B, H, T, T]
            scores = scores + (mask * -1e9)

        attn = tf.nn.softmax(scores, axis=-1)
        out = tf.matmul(attn, V)          # [B, H, T, depth]

        # gabung head: [B, T, d_model]
        out = tf.transpose(out, perm=[0, 2, 1, 3])
        B, T, _, _ = tf.unstack(tf.shape(out))
        out = tf.reshape(out, (B, T, self.d_model))

        # proyeksi output
        out = tf.matmul(out, self.Wo)
        return out, attn


## 🧱 Feed-Forward Network Posisi

**Theory**: Setiap posisi urutan dilewatkan melalui MLP dua lapis yang sama (position-wise FFN) untuk menambah kapasitas non-linear.

$$
\text{FFN}(x) = W_2 \,\text{ReLU}(x W_1 + b_1) + b_2
$$


In [6]:
class PositionwiseFFN(layers.Layer):
    def __init__(self, d_model, d_ff, **kwargs):
        super().__init__(**kwargs)
        self.dense1 = layers.Dense(d_ff, activation="relu")
        self.dense2 = layers.Dense(d_model)

    def call(self, x):
        return self.dense2(self.dense1(x))


In [7]:
class EncoderBlock(layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, **kwargs):
        super().__init__(**kwargs)
        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = PositionwiseFFN(d_model, d_ff)
        self.ln1 = layers.LayerNormalization(epsilon=1e-6)
        self.ln2 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, x, mask=None, training=False):
        # self-attention + residual + norm
        attn_out, _ = self.mha(x, mask)
        x = self.ln1(x + attn_out)
        # ffn + residual + norm
        ffn_out = self.ffn(x)
        x = self.ln2(x + ffn_out)
        return x


In [8]:
vocab_size = 5000
d_model = 128
num_heads = 4
d_ff = 512
max_len = 50

# input: urutan ID token
inputs = layers.Input(shape=(max_len,))
x = layers.Embedding(vocab_size, d_model)(inputs)

# (opsional) positional encoding bisa ditambahkan di sini

x = EncoderBlock(d_model, num_heads, d_ff)(x)   # 1 layer encoder

# pooling sederhana (ambil vektor posisi [CLS]-like: sini pakai mean)
x = tf.reduce_mean(x, axis=1)
outputs = layers.Dense(2, activation="softmax")(x)  # mis. klasifikasi biner

mini_transformer = tf.keras.Model(inputs=inputs, outputs=outputs)
mini_transformer.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

mini_transformer.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 50)]              0         
                                                                 
 embedding (Embedding)       (None, 50, 128)           640000    
                                                                 
 encoder_block (EncoderBlock  (None, 50, 128)          197760    
 )                                                               
                                                                 
 tf.math.reduce_mean (TFOpLa  (None, 128)              0         
 mbda)                                                           
                                                                 
 dense_2 (Dense)             (None, 2)                 258       
                                                                 
Total params: 838,018
Trainable params: 838,018
Non-trainable

## ✅ Ringkasan Chapter 5

**Theory**: Transformer menggantikan RNN/CNN pada banyak tugas NLP dengan mengandalkan self-attention penuh pada urutan input, encoder–decoder, dan multi-head attention.

Keras sub-classing memudahkan implementasi blok utama (self-attention, multi-head, FFN, encoder/decoder block) yang dapat disusun menjadi model Transformer end-to-end untuk tugas seperti machine translation.
